In [1]:
from rag_model_test import RAGRecrutamento
from preprocessing import TextPreprocessing
import settings
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json(settings.GOOGLE_CREDENTIALS)

c:\Users\willb\Documents\Projetos\DataThon-Fiap\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\willb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
full_df = client.list_rows("datathon-473001.datathon.full_data").to_dataframe()


c:\Users\willb\Documents\Projetos\DataThon-Fiap\venv\lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [3]:
full_df.shape

(45071, 38)

In [4]:
full_df_filt = full_df[[
  'perfil_vaga_estado',
  'perfil_vaga_cidade',
  'perfil_vaga_nivel_profissional',
  'perfil_vaga_nivel_academico',
  'perfil_vaga_nivel_ingles',
  'perfil_vaga_nivel_espanhol',
  'perfil_vaga_principais_atividades',
  'perfil_vaga_competencia_tecnicas_e_comportamentais',
  'perfil_vaga_demais_observacoes',
  'infos_basicas_objetivo_profissional',
  'infos_basicas_local',
  'formacao_e_idiomas_nivel_academico',
  'formacao_e_idiomas_nivel_ingles',
  'formacao_e_idiomas_nivel_espanhol',
  'cv_pt_0',
  'situacao_candidado']]

In [5]:
# Função de tratamento
def tratar_textos(lista_textos):
    processamento = TextPreprocessing()
    textos_validos = [str(t).strip() for t in lista_textos if pd.notna(t)]
    texto = processamento.preprocess_text(texts=textos_validos,
                                          apply_lower=True,
                                          remove_ponctuation=True,
                                          remove_numbers=True,
                                          clean_html=True,
                                          apply_unidecode=True,
                                          remove_stopwords=True,
                                          remove_duplicates=True,
                                          apply_stemming=False,
                                          apply_lemmitization=False,
                                          )
    return texto


In [6]:
colunas_vaga = [
    'perfil_vaga_estado',
  'perfil_vaga_cidade',
  'perfil_vaga_nivel_profissional',
  'perfil_vaga_nivel_academico',
  'perfil_vaga_nivel_ingles',
  'perfil_vaga_nivel_espanhol',
  'perfil_vaga_principais_atividades',
  'perfil_vaga_competencia_tecnicas_e_comportamentais',
  'perfil_vaga_demais_observacoes'
]

In [7]:
colunas_candidato = [
  'infos_basicas_objetivo_profissional',
  'infos_basicas_local',
  'formacao_e_idiomas_nivel_academico',
  'formacao_e_idiomas_nivel_ingles',
  'formacao_e_idiomas_nivel_espanhol',
  'cv_pt_0'
]

In [8]:
# aplica só nas colunas escolhidas
full_df['texto_tratado_vaga'] = full_df[colunas_vaga].apply(
    lambda row: tratar_textos(row.tolist()), axis=1
)

# aplica só nas colunas escolhidas
full_df['texto_tratado_candidato'] = full_df[colunas_candidato].apply(
    lambda row: tratar_textos(row.tolist()), axis=1
)

# exemplo: pegar junto com a situacao do candidato
df_final = full_df[['situacao_candidado', 'texto_tratado_vaga', 'texto_tratado_candidato']]

In [9]:
df_final.situacao_candidado.value_counts()

situacao_candidado
prospect                          16602
encaminhado ao requisitante       13807
inscrito                           3183
não aprovado pelo cliente          2964
contratado pela decision           2254
desistiu                           2061
não aprovado pelo rh               1583
não aprovado pelo requisitante      653
entrevista técnica                  485
entrevista com cliente              363
sem interesse nesta vaga            361
em avaliação pelo rh                339
aprovado                            183
contratado como hunting             172
desistiu da contratação              48
documentação pj                       4
documentação clt                      3
documentação cooperado                2
recusado                              2
proposta aceita                       1
encaminhar proposta                   1
Name: count, dtype: int64

In [10]:
map_situacao = {
    # --- Pipeline / Em processo ---
    "prospect": "pipeline",
    "inscrito": "pipeline",
    "encaminhado ao requisitante": "pipeline",
    "entrevista técnica": "pipeline",
    "entrevista com cliente": "pipeline",
    "em avaliação pelo rh": "pipeline",
    "documentação pj": "pipeline",
    "documentação clt": "pipeline",
    "documentação cooperado": "pipeline",
    "encaminhar proposta": "pipeline",

    # --- Reprovados / Desistentes ---
    "não aprovado pelo cliente": "reprovado",
    "não aprovado pelo rh": "reprovado",
    "não aprovado pelo requisitante": "reprovado",
    "desistiu": "reprovado",
    "desistiu da contratação": "reprovado",
    "sem interesse nesta vaga": "reprovado",
    "recusado": "reprovado",

    # --- Aprovados / Contratados ---
    "aprovado": "aprovado",
    "proposta aceita": "aprovado",
    "contratado pela decision": "aprovado",
    "contratado como hunting": "aprovado"
}

In [11]:
df_final = (df_final.assign(
    situacao_candidato_grp=lambda df_: df_["situacao_candidado"].map(map_situacao)
).drop('situacao_candidado', axis=1)
)

In [12]:
rag_model = RAGRecrutamento()

In [13]:
job_description = df_final.texto_tratado_vaga.iloc[350]

curriculo = df_final.texto_tratado_candidato.iloc[350]

In [14]:
resultado = rag_model.rag(curriculo, job_description)

In [15]:
curriculo

'crescimento profissional exemplo atuacao area aprender sempre maringa parana ensino superior cursando basico experiencia empresa associacao amparo pessoas idosas wajunkai data admissao saida atualmente ano mes semprebom industria produtos naturais meses atribuicoes realizacoes recrutamento selecao rescisao lancamentos fechamento folha pagamento ferias salario banco horas conferencia apuracao ponto controle pedido beneficios atendimento fornecedores lancamento motoristas atender orientar colaboradores acordo normas suporte faturamento correcao pedidos comissao santa rita saude s anos telefonica brasil telefonico receptivo retencao clientes confirmacao cadastro contratos acompanhamento comprometimento metas animakids eventos jhm telecom ltda influx school idiomas analise processos planejamento estrategico vendas cursos contas receber manutencao aluno conferir contrato acompanhar agenda pedagogica tutores presencial mostrar espaco escola elaborar emissao boleto outras atividades administ

In [16]:
job_description

'sao paulo senior ensino superior completo basico analista recrutamento selecao tech recruiter conhecimentos etapas processo triagem cv candidatos contato habilidade pacote office boa desenvoltura comunicacao vontade crescimento profissional conhecimento sobre linguagens programacao ferramentas bibliotecas boas praticas avaliacao tecnica vagas tecnologia rh necessario curso nivel agilidade contratacao profissionais vaga urgente'

In [17]:
print(resultado)

```json
{
  "pontos_de_destaque": [
    "Experiência em ensino superior, demonstrando conhecimento de linguagens de programação e ferramentas bibliotecas.",
    "Demonstração de habilidades analíticas e de avaliação técnica através de processos de recrutamento e seleção.",
    "Comunicação eficaz e proatividade na interação com candidatos e colaboradores.",
    "Vontade de crescimento profissional e busca por novos desafios na área de tecnologia.",
    "Aprovação em cursos de agilidade e profissionalismo, demonstrando compromisso com a área."
  ],
  "perguntas_entrevista": [
    "Com base na sua experiência em ensino superior, como você lida com candidatos com diferentes níveis de conhecimento?",
    "Descreva uma situação em que você precisou analisar um currículo e identificar as qualificações mais relevantes para uma vaga. Quais foram os critérios que você utilizou?",
    "Como você se mantém atualizado sobre as tendências da tecnologia e as ferramentas que são relevantes para a áre